In [18]:
%%capture
import pandas as pd
import numpy as np
import os
import re
import gensim
from gensim.models import Word2Vec
import nltk
from nltk import RegexpTokenizer as rpt
from nltk.corpus import stopwords as sw
from nltk import tokenize    
from string import punctuation 

nltk.download('punkt')
nltk.download('stopwords')
stopwords = sw.words('portuguese')

data_url="https://raw.githubusercontent.com/liraop/recinfo_lab2/master/data/results.csv"
data = pd.read_csv(data_url).replace(np.nan, '', regex=True)

def parse(text):
    words = []
    word_pattern = rpt(r'\w+')
    
    patterns = [word_pattern]
    
    for pattern in patterns:
        tokens = []
        for token in pattern.tokenize(text):
            if token not in stopwords and len(token) > 3:
                tokens.append(token)
        words.extend(tokens)
    return words

def build_sentences(dataset):
    sentences = []
    
    for entry in dataset.title:            
        title = []
        for ngram in parse(entry):
            title.append(ngram)
        
        sentences.append(title)
    return sentences

titles = build_sentences(data)

In [17]:
titles

[['sociedade', 'Rubens', 'Paiva', 'facínoras', 'mataram'],
 ['Justiça',
  'suspende',
  'decisão',
  'proibia',
  'Forças',
  'Armadas',
  'celebrarem',
  'golpe',
  '1964'],
 ['Governo',
  'Bolsonaro',
  'prega',
  'negacionismo',
  'histórico',
  'sobre',
  'ditadura'],
 ['Quando', 'pais', 'Gabo', 'perceberam', 'filho', 'mentiroso'],
 ['Rádios',
  'canadenses',
  'banem',
  'músicas',
  'Michael',
  'Jackson',
  'programação'],
 ['Rosângela',
  'crianças',
  'sequestradas',
  'famílias',
  'militares',
  'ditadura'],
 ['Lógica', 'usar', 'torturadores', 'ditadura', 'crime', 'usada', 'milícias'],
 ['Bolsonaro',
  'troca',
  'embaixada',
  'escritório',
  'Jerusalém',
  'evita',
  'retaliação',
  'palestina'],
 ['lado', 'sombrio', 'Dickens'],
 ['amor', 'mulheres', 'sacudiu', 'Paraguai'],
 ['feminista', 'Susan', 'Sontag', 'contra', 'Norman', 'Mailer'],
 ['imagens', 'questionadoras', 'Harun', 'Farocki'],
 ['manuscrito',
  'anos',
  'solidão',
  'García',
  'Márquez',
  'acreditava',
  'pe

In [20]:
model = Word2Vec(titles, min_count=1)
print(model)

Word2Vec(vocab=28, size=100, alpha=0.025)
